![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [66]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "../modulo 17/input/features.txt"
filename_labels = "../modulo 17/input/activity_labels.txt"

filename_subtrain = "../modulo 17/input/train/subject_train.txt"
filename_xtrain = "../modulo 17/input/train/X_train.txt"
filename_ytrain = "../modulo 17/input/train/y_train.txt"

filename_subtest = "../modulo 17/input/test/subject_test.txt"
ffilename_xtest = "../modulo 17/input/test/X_test.txt"
filename_ytest = "../modulo 17/input/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'],  sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

In [94]:
X_train.dropna(axis=1, inplace=True)
X_test.dropna(axis=1, inplace=True)

In [97]:
print("Número de amostras em X_train:", X_train.shape[0])
print("Número de amostras em y_train:", y_train.shape[0])
print("Número de amostras em X_test:", X_test.shape[0])
print("Número de amostras em y_test:", y_test.shape[0])

# Verificar se há linhas duplicadas nos dados de entrada
print("Linhas duplicadas em X_train:", X_train.duplicated().sum())
print("Linhas duplicadas em X_test:", X_test.duplicated().sum())


Número de amostras em X_train: 7352
Número de amostras em y_train: 7352
Número de amostras em X_test: 2947
Número de amostras em y_test: 2947
Linhas duplicadas em X_train: 0
Linhas duplicadas em X_test: 0


In [96]:
print("NAN em X_train:", X_train.isna().sum())
print("NAN em X_test:", X_test.isna().sum())

NAN em X_train: 1 tBodyAcc-mean()-X                         0
2 tBodyAcc-mean()-Y                         0
3 tBodyAcc-mean()-Z                         0
4 tBodyAcc-std()-X                          0
5 tBodyAcc-std()-Y                          0
                                           ..
557 angle(tBodyGyroMean,gravityMean)        0
558 angle(tBodyGyroJerkMean,gravityMean)    0
559 angle(X,gravityMean)                    0
560 angle(Y,gravityMean)                    0
561 angle(Z,gravityMean)                    0
Length: 561, dtype: int64
NAN em X_test: 1 tBodyAcc-mean()-X                         0
2 tBodyAcc-mean()-Y                         0
3 tBodyAcc-mean()-Z                         0
4 tBodyAcc-std()-X                          0
5 tBodyAcc-std()-Y                          0
                                           ..
557 angle(tBodyGyroMean,gravityMean)        0
558 angle(tBodyGyroJerkMean,gravityMean)    0
559 angle(X,gravityMean)                    0
560 angle(Y,gravityMean

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [98]:
%%time
clf = DecisionTreeClassifier(random_state=1993, ccp_alpha=0.001).fit(X_train, y_train)

CPU times: total: 8.27 s
Wall time: 10.6 s


In [99]:
print(f'Acuracia base treino: {clf.score(X_train, y_train)}')
print(f'Acuracia base teste: {clf.score(X_test, y_test)}')

Acuracia base treino: 0.9757889009793254
Acuracia base teste: 0.8805564981336953


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [100]:
%%time
prcomp = PCA(n_components=1).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

print(f'Dimensões da base de treinamento: {pc_treino.shape}')
print(f'Dimensões da base de teste: {pc_teste.shape}')


clf = DecisionTreeClassifier(ccp_alpha=0.001)
clf.fit(pc_treino, y_train)

print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 0.499727965179543
Acurácia na base de teste: 0.45707499151679676

CPU times: total: 312 ms
Wall time: 262 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [104]:
%%time
comp = [1,2,5,10,50]
for x in comp:
    prcomp = PCA(n_components=x).fit(X_train)

    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)

    print(f'Dimensões da base de treinamento: {pc_treino.shape}')
    print(f'Dimensões da base de teste: {pc_teste.shape}')


    clf = DecisionTreeClassifier(ccp_alpha=0.001)
    clf.fit(pc_treino, y_train)

    print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
    print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 0.499727965179543
Acurácia na base de teste: 0.45707499151679676

Dimensões da base de treinamento: (7352, 2)
Dimensões da base de teste: (2947, 2)
Acurácia na base de treinamento: 0.6127584330794341
Acurácia na base de teste: 0.5846623685103495

Dimensões da base de treinamento: (7352, 5)
Dimensões da base de teste: (2947, 5)
Acurácia na base de treinamento: 0.8460282916213275
Acurácia na base de teste: 0.7885985748218527

Dimensões da base de treinamento: (7352, 10)
Dimensões da base de teste: (2947, 10)
Acurácia na base de treinamento: 0.8932263329706203
Acurácia na base de teste: 0.823210044112657

Dimensões da base de treinamento: (7352, 50)
Dimensões da base de teste: (2947, 50)
Acurácia na base de treinamento: 0.9178454842219804
Acurácia na base de teste: 0.823549372242959

CPU times: total: 2.75 s
Wall time: 2.8 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

###### O que aconteceu com a acurácia?
<blockquote>Conforme se aumenta o numero de componentes a acuracia também aumenta, começando com 0.45 com 1 componente e chegando a .82 com 50 componentes</blockquote>

###### O que aconteceu com o tempo de processamento?
<blockquote>O tempo caiu de 10.6 usando todos os componentes para 2.8 s para a execução de todas as 5 variações de componentes. podemos obter resultados satisfátorios usando a técnica $PCA$ semelhantes aos resultados com todos os componenetes do dataframe mas com um tempo muito menor e usando menos recursos computacionais.
</blockquote>